In [1]:
response = {
    "articles": [
        {
            "title": "Prime Minister inaugurates the $400 temple in Kanpur"
        },
        {
            "title": "President launches new initiative"
        }
    ]
}

In [2]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

def get_main_verb(sentence):
    doc = nlp(sentence)
    main_verb = None
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ != "aux":
            main_verb = token.text
            break
    return main_verb

In [3]:
from DataModelling import DataModel

data_model = DataModel()


for article in response["articles"]:
    title = article["title"]
    main_verb = get_main_verb(title)
    custom_pattern = re.compile(fr'(?P<entity1>.+?)\s+(?P<relation>\b(?:{main_verb})\b)\s+(?P<entity2>.+)')
    match = custom_pattern.match(title)
    entity1 = match.group("entity1").strip()
    entity2 = match.group("entity2").strip()
    relation = match.group("relation").strip()
    data_model.add_entity(entity1)
    data_model.add_entity(entity2)
    data_model.add_relation(relation, [
        entity1,
        entity2
    ])
    data_model.save_to_csv()
    print(f"Added {entity1} {relation} {entity2} to the data model")

Added Prime Minister inaugurates the $400 temple in Kanpur to the data model
Added President launches new initiative to the data model
